Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Build TensorFlow Lite and Select-tf-ops with Bazel 

In [ ]:
# Create folders
!mkdir -p '/android/sdk'

# Download and move android SDK tools to specific folders
!wget -q 'https://dl.google.com/android/repository/tools_r25.2.5-linux.zip'

!unzip 'tools_r25.2.5-linux.zip'
!mv '/content/tools' '/android/sdk'
# Copy paste the folder
!cp -r /android/sdk/tools /android/android-sdk-linux

# Download NDK, unzip and move contents
!wget 'https://dl.google.com/android/repository/android-ndk-r19c-linux-x86_64.zip'

!unzip 'android-ndk-r19c-linux-x86_64.zip'
!mv /content/android-ndk-r19c /content/ndk
!mv '/content/ndk' '/android'
# Copy paste the folder
!cp -r /android/ndk /android/android-ndk-r19c

# Remove .zip files
!rm 'tools_r25.2.5-linux.zip'
!rm 'android-ndk-r19c-linux-x86_64.zip'

# Make android ndk executable to all users
!chmod -R go=u '/android'

In [ ]:
# Clone TensorFlow repository
!git clone https://github.com/tensorflow/tensorflow.git

In [ ]:
# Set and view environment variables
%env PATH = /usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/android/sdk/tools:/android/sdk/platform-tools:/android/ndk
%env ANDROID_SDK_API_LEVEL=29
%env ANDROID_API_LEVEL=29
%env ANDROID_BUILD_TOOLS_VERSION=29.0.2
%env ANDROID_DEV_HOME=/android
%env ANDROID_NDK_API_LEVEL=21
%env ANDROID_NDK_FILENAME=android-ndk-r19c-linux-x86_64.zip
%env ANDROID_NDK_HOME=/android/ndk
%env ANDROID_NDK_URL=https://dl.google.com/android/repository/android-ndk-r19c-linux-x86_64.zip
%env ANDROID_SDK_FILENAME=tools_r25.2.5-linux.zip
%env ANDROID_SDK_HOME=/android/sdk
#%env ANDROID_HOME=/android/sdk
%env ANDROID_SDK_URL=https://dl.google.com/android/repository/tools_r25.2.5-linux.zip

#!echo $PATH
!export -p

In [ ]:
# Update sdk, tools etc
!android update sdk --no-ui -a \
  --filter tools,platform-tools,android-29,build-tools-29.0.2

## Install BAZEL

In [ ]:
# Download Latest version of Bazelisk
!wget https://github.com/bazelbuild/bazelisk/releases/latest/download/bazelisk-linux-amd64

# Make script executable
!chmod +x bazelisk-linux-amd64

# Adding to the path
!sudo mv bazelisk-linux-amd64 /usr/local/bin/bazel

# Extract bazel info
!bazel

In [ ]:
# Move tensorflow folder
!mv /content/tensorflow /tensorflow
# Rename it
!mv /tensorflow /tensorflow_src

In [ ]:
# Move inside tensorflow folder and run configure
%cd /tensorflow_src
!configs=('/usr/bin/python3''/usr/lib/python3/dist-packages''N''N''N''N''-march=native -Wno-sign-compare''y''/android/sdk')

# Print some info
!printf '%s\n' "${configs[@]}" | ./configure

In [ ]:
# Pull the latest code from tensorflow.
#!git pull -a
#!git checkout ${FLAG_CHECKPOINT}

In [ ]:
# Configure Bazel
#!source tensorflow/tools/ci_build/release/common.sh

# Build TensorFlow Lite 

In [ ]:
!ls

In [ ]:
!bazel build -c opt --cxxopt='--std=c++14' \
    --config=monolithic \
    --fat_apk_cpu=x86,x86_64,arm64-v8a,armeabi-v7a \
    --host_crosstool_top=@bazel_tools//tools/cpp:toolchain \
    //tensorflow/lite/java:tensorflow-lite

## Build select-tf-ops

To build custom TensorFlow Lite library and tensorflow-lite-select-tf-ops.aar file navigate to /tensorflow_src/tensorflow/lite/tools and add file https://github.com/farmaker47/Build_TensorFlow_Lite_Support_Library_With_Docker/blob/main/build_select_tf_ops_aar.sh( build_select_tf_ops_aar.sh) Then execute it.

For param input_models give the path of the tflite model that contains special ops. 

With this procedure the final 2 .aar files are of smaller size and can be used inside an android studio project like:
https://www.tensorflow.org/lite/guide/build_android#add_aar_directly_to_project

In [ ]:
!bash /tensorflow_src/tensorflow/lite/tools/build_select_tf_ops_aar.sh \
  --input_models='/content/BasicTokenizerModel.tflite' \
  --target_archs=arm64-v8a,armeabi-v7a